In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio
#!pip install torchsummary

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # To have better Traceback if CUDA fails

import os
import csv
import time
import logging
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_value = 2020
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)


Bad key "text.kerning_factor" on line 4 in
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
UNITS = 256
VOCAB_SIZE = 30000
LIMIT = 1000
N_FRAMES = 400
N_EPOCHS = 1
params = {'batch_size': None,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}

logging.basicConfig(filename='mytraining.log', format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

# Initializing the datasets for training and validation
training_set = LibriSpeechDataset(limit=LIMIT, n_frames=N_FRAMES, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=LIMIT, n_frames=N_FRAMES, version='dev-clean')

In [5]:
# Let's see a single utterance

LibriSpeechDataset(limit=LIMIT, n_frames=400, version='train-clean-360')[0]

(tensor([[[2.9673e-05, 2.9673e-05, 2.9673e-05,  ..., 1.4835e-04,
           1.4388e-04, 1.3941e-04],
          [4.4498e-06, 4.4498e-06, 4.4498e-06,  ..., 4.0891e-03,
           3.1622e-03, 2.2350e-03],
          [4.5445e-06, 4.5469e-06, 4.5852e-06,  ..., 1.8744e-02,
           1.7684e-02, 1.6625e-02],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]]),
 tensor([  101,   101,  2045,  2001,  2625,  1997,  1037,  4306,  2055,  2085,
          1998, 14163,  6894,  5289,  2106,  2025,  2031,  2000,  2562,  2067,
          1037,  5481,  2004,  2002,  2441,  1996,  9445,  3460,  6031,  2988,
          1996, 14460,  4214,  8721,  2000, 10767,  1998, 27468,  1996,  2221]))

In [6]:
# Defining an encoder, decoder optimizers and criterion
encoder = EncoderCONV2DRNN(device=device, batch_sz=params['batch_size'], hidden_size=UNITS, spec_dim=(1, 39, 400)).to(device)
decoder = DecoderATTRNN(vocab_size=VOCAB_SIZE,  batch_sz=params['batch_size'], dec_units=UNITS, hidden_size=UNITS, 
                        encoder_timestamp=encoder.encoder_timestamp).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [7]:
# Train the model

global_trainer(N_EPOCHS, training_set, dev_set, params, encoder, decoder, encoder_optimizer,
               decoder_optimizer, criterion, device)

 ====== ====== ====== ====== ====== ======
      The model has 24648263 parameters
 ====== ====== ====== ====== ====== ======


Epoch        1: 100%|████████████████████| 62/62 [04:52<00:00,  4.72s/it, Train loss 6.9593 Eval loss 4.4181]

Time taken for the training 0.081941 hours



In [ ]:
mfccs, references = training_set[1]
tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
references = [tokenizer.convert_ids_to_tokens(ind) for ind in references.numpy().tolist()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang_tokenizer=tokenizer, 
          device=device, beam_search=True, beam_width=10)

In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU


In [ ]:
#!pip freeze > recquirements.txt